# Neural Ordinary Differential Equations
A discussion of an interesting new paradigm in Neural Networks, adapted from "Neural Ordinary Differential Equations" by Chen et al. 2019 (https://arxiv.org/pdf/1806.07366.pdf)

## 1 - Background 

### 1.1 - Neural Network Setup
Neural networks are a set of algorithms that are loosely based on the functonality of the human brain. These algorithms are designed to learn patterns in numerical data, by propagating input data through a series of hidden layers, with each layer being composed of several artificial 'neurons' or nodes. The final result is the output layer, a numerical representation of our desired prediction.

Consider an input dataset composed of a set of $m$ features: $(\vec{x_1}, \vec{x_2}, ..., \vec{x_m})$ . <br>
We add a bias component to this dataset, $\vec{x_0} = 1$, a vector of ones, yielding our input layer, $\vec{x} := \vec{x_0}, ... \vec{x_m}$. <br>
We need to now consider how to pass this input to our first hidden layer. Suppose this the first layer is composed of $n$ nodes, we define a set of weights $w=\vec{w_1},\vec{w_2} ..., \vec{w_n}$ where each $\vec{w_i}$ is a vector of length $m$ corresponding to node $i$. We then define $\vec{z_i} := \vec{x}\cdot \vec{w_{i}}$. <br>

We are passing $\vec{z_i}$ to each node in the first hidden layer, but but what occurs at the node, and what is output at the next layer? The answer is the activation function. For the purpose of this discussion, we will focus solely on the sigmoid activation function $\sigma(z) = \dfrac{1}{1+e^{-z}}$, though it should be noted that many others exist. At each node, we will compute $\sigma(z_i)$ and that's it! We have our single layer representation. <br>

To extrapolate to a full neural network, we simply repeat this process, using the output of the activation functions of the previous layer as the new input layer, taking the dot product with a new set of weights and computing the activation of these new inputs. The final layer will be our output layer, $\vec{y}$, and will have a number of neurons corresponding to our desired predictable (if we are predicting hand-written digits as in the exemplary MNIST dataset, our $\vec{y}$ will be a vector of length 10). <br>

Our problem is thus clear, we must compute: $W$, the weights, $L$, the number of layers and $n_{}$ the number of neurons in each layer. And we must do so efficiently, as there will often be many many weights to compute and many layers in our network. 

### 1.2 - Computing parameters and hyperparameters
We will seperate the values we wish to compute into two categories: 'parameters' (the weights) and the 'hyperparameters' (the number of layers and the number of neurons in each layer). In general, determining the hyperparameters is much more of an art than a science, and requires significant experimentation and intuition. On the other hand, determining the parameters can be done by following one of several well-defined protocols. <br>
For our purposes, we will simply define a cost-function, which will turn out to be an aggregate of the cost function used in typical logistic regression (__NOTE: if you don't know logistic regression, stop here and go learn that first__). For now, I will simply state the cost function of a neural network implementing the sigmoid activation function: <br> 
\begin{align}
J(\Theta) = - \dfrac{1}{m}\sum_{i=1}^m \sum_{k=1}^K\left[
y_{k}^{(i)} \log((h_{\Theta}(x^{(i)}))_k) + (1-y)\log(1-(h_{\Theta}(x^{(i)}))_k) 
\right]
+ \dfrac{\lambda}{2m}\sum_{l=1}^{L-1}\sum_{i=1}^{s_l}\sum_{j=1}^{s_{l+1}}\left(
\Theta_{j,i}^{l}
\right)^{2}
\end{align}
where $L$ is the total number of layers in the network, $s_l$ is the number of nodes in layer $l$, $K$ is the number of output classes/units and $\lambda$ is a regularization parameter, to prevent overfitting (another hyperparameter to compute). 

With this cost function, we will choose starting values for the parameters, feed these forward through the network, and calculate the loss, known as the feedforward step. We then take this loss to update the weight and bias values moving backward through the network, known as the backpropagation step. 

## 2 - Neural Networks in Python

#### 2.1 - Definitions 
As per [this](https://towardsdatascience.com/how-to-build-your-own-neural-network-from-scratch-in-python-68998a08e4f6) tutorial, we will define the input layer $\textbf{x}$, an arbitrary amount of hidden layers, an output layer $\hat{\textbf{y}}$, the set of weights $\textbf{W}$ and biases $\textbf{b}$ and the activation function $\sigma$.

We will now create this class in Python, for a neural network with one hidden layer, having 4 nodes:

In [3]:
import numpy as np

class NeuralNetwork:
    def __init__(self, x, y):
        self.input      = x
        self.weights1   = np.random.rand(self.input.shape[1],4) 
        self.weights2   = np.random.rand(4,1)                 
        self.y          = y
        self.output     = np.zeros(y.shape)

To this class definition, we will add a feedforward step, where we assume the bias values $\textbf{b}$ are $0$ for simplicity.

### 2.2 - Training
To train this network, we need to find appropriate values for $\textbf{W}$ and $\textbf{b}$. We do so by feeding forward through the network with an initial set of parameters, and subsequently updating these parameters using backpropagation with an appropriate cost function. Adding the feedforward to our class definition is trivial:

In [5]:
class NeuralNetwork:
    
    def __init__(self, x, y):
        self.input      = x
        self.weights1   = np.random.rand(self.input.shape[1],4) 
        self.weights2   = np.random.rand(4,1)                 
        self.y          = y
        self.output     = np.zeros(self.y.shape)

    def feedforward(self):
        self.layer1 = sigmoid(np.dot(self.input, self.weights1))
        self.output = sigmoid(np.dot(self.layer1, self.weights2))

Now, to add the backpropagation step, we must determine the gradient of our cost function. 

### 2. - MNIST Dataset
We will now work towards building a standard neural network from scratch, in Python, using the MNIST handwritten number dataset (a toy dataset commonly used in teaching image recognition).